

I have a multidimensional array, for example four dimensions 
with the first dimension representing the state variables and 
the other dimensions represent spatial dimensions.

```
Eps = np.array((n_b, n_I, n_J, n_K), np.float_) 
```

and another array with spatial dimensions represents the reversibility 
criterion

```
f = np.array((n_I, n_J, n_K), np.float_)
```

To handle the inelastic part of the state I extract the index map
using the where function

```
I = np.where(f > 0)
```
Now I want to access the inelastic elements in the Eps array. I know I can do 
in this way:
```
Eps_ = np.moveaxis(Eps, 0, -1)
Eps_[I]
```
I wonder if I can do this without `np.moveaxis`. The necessity to rearrange the dimensions complicates the usage of lambdified functions which expect the 
dimension of state variables `n_b` to come in the first position.  

Here is the summary of the code once again. I am searching for index operator 
for the array `Eps` which selects the same elements as `Eps_` so that the first 
dimension remains and the resulting array can be submitted to the 
lambdified function exactly in the same way as the whole original array `Eps`.

```
import numpy as np
n_b, n_I, n_J, n_K = 3, 4, 5, 6
Eps = np.zeros((n_b, n_I, n_J, n_K), np.float_) - 1
f = np.sum(Eps, axis=0)
f[0, 1, 2] = 1
f[0, 2, 2] = 1
f[0, 0, 2] = 1
f[0, 0, 0] = 1
I = np.where(f > 0)
Eps_ = np.moveaxis(Eps, 0, -1)
Eps_[I]
```

Sure, let's dive deeper into how the solution works and provide additional context and explanation.



Given:

- `Eps` is a 4-dimensional array with the shape `(n_b, n_I, n_J, n_K)`.

- `f` is a 3-dimensional array with the shape `(n_I, n_J, n_K)`.



You used `np.where(f > 0)` to get the indices `I` where the condition `f > 0` is met. This results in a tuple of arrays, each representing the indices along each of the spatial dimensions.



To access the corresponding elements in `Eps` without changing the order of dimensions, we can use advanced indexing directly. Here's the full example:



```python

import numpy as np



# Initialize dimensions

n_b, n_I, n_J, n_K = 3, 4, 5, 6



# Initialize arrays

Eps = np.zeros((n_b, n_I, n_J, n_K), np.float_) - 1

f = np.sum(Eps, axis=0)



# Set some criteria in f

f[0, 1, 2] = 1

f[0, 2, 2] = 1

f[0, 0, 2] = 1

f[0, 0, 0] = 1



# Find indices where f > 0

I = np.where(f > 0)



# Adjust I to have the proper shape for indexing Eps directly

I_full = (slice(None), *I)



# Access the elements in Eps

Eps_selected = Eps[I_full]



print(Eps_selected)

```



In the code above, the key point is the creation of `I_full = (slice(None), *I)`:



- `slice(None)` indicates that we want all elements along the first dimension (`n_b`). This is similar to `:` in array indexing.

- `*I` unpacks the tuple `I` so that each index array corresponds to each of the spatial dimensions (`n_I`, `n_J`, `n_K`).



Using `I_full` to index `Eps` directly preserves the first dimension while selecting the appropriate elements from the spatial dimensions. This way, the shape of the resulting `Eps_selected` array will be `(n_b, num_selected_elements)`, where `num_selected_elements` is the number of elements that met the condition `f > 0`.



Finally, you can pass `Eps_selected` to the lambdified function or perform further operations as needed, without rearranging the dimensions in `Eps`.

In [ ]:
import numpy as np

def get_f_lambdified(Eps):
    u, v, w = Eps
    return np.sqrt(u**2 + v**2 + w**2)

n_b, n_I, n_J, n_K = 3, 4, 5, 6
Eps = np.zeros((n_b, n_I, n_J, n_K), np.float_) - 1
f = np.sum(Eps, axis=0)
f[0, 1, 2] = 1
f[0, 2, 2] = 1
f[0, 0, 2] = 1
f[0, 0, 0] = 1
I = np.where(f > 0)
bI = (slice(None), *I)
f[I].shape, Eps[bI].shape

In [ ]:
f_Eps = get_f_lambdified(Eps)
f_Eps.shape

In [ ]:
f_Eps_I = get_f_lambdified(Eps[bI])
f_Eps_I.shape

In [ ]:
# 1D
import numpy as np

def get_f_lambdified(Eps):
    u, v, w = Eps
    return np.sqrt(u**2 + v**2 + w**2)

n_b, n_I = 3, 1
Sig_k = np.zeros((n_b, n_I), np.float_) - 1
f_k = np.sum(Eps, axis=0)
f[0] = 1
I = f > 0
bI = (slice(None), *I)

print('f_k', f_k.shape) 
print('Sig_k', Sig_k.shape)

print('I', I)
print('bI', bI)

print('f_k[I]', f_k[I].shape) 
print('Sig_k[bI]', Sig_k[bI].shape)

f_Sig_I = get_f_lambdified(Sig_k[bI])
f_Sig_I.shape